# 1.不包含可训练权重的简化版自注意力机制 

In [2]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

## 1.1 以计算inputs中"journey"的context vector为例

In [ ]:
'''Step1: Get the attention scores'''
query = inputs[1] #这里的query就是Transformers里面的query，就是当前待计算context vector的元素的embedding vector

attn_scores_1 = torch.matmul(query, inputs.T) #这里因为是将一个形状为(1,3)的向量与形状为(6,3)的矩阵相乘，所以需要对矩阵进行转置操作

attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query) #这里现在进行点积的是两个一维向量，就不用进行转置操作

print(attn_scores_1)
print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])
tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [6]:
'''Simulate the dot product operation'''
res = 0

for idx, element in enumerate(inputs[0]):
    res += inputs[0][idx] * query[idx]

print(res)
print(torch.dot(inputs[0], query))

tensor(0.9544)
tensor(0.9544)


In [10]:
'''Step 2: Normaliza the attention scores to get the final attention weights'''
#1.A simple way
attn_weights_2_tmp = attn_scores_2  / attn_scores_2.sum()
print(attn_weights_2_tmp)
print(attn_weights_2_tmp.sum())

#2.Using softmax function 
# Using self-implemented softmax function
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim = 0)

attn_weights_2_naive = softmax_naive(attn_scores_2)
print(attn_weights_2_naive)
print(attn_weights_2_naive.sum())

# Using torch.softmax()
attn_weights_2 = torch.softmax(attn_scores_2, dim = 0)
print(attn_weights_2)
print(attn_weights_2.sum())

tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
tensor(1.0000)
tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
tensor(1.)
tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
tensor(1.)


In [11]:
'''Step 3: Comput the context vector '''
query = inputs[1]

context_vec_2 = torch.zeros(query.shape) #因为context vector的shape和token embedding vector是一样的
for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i] * x_i #就是将序列中每个token的注意力权重乘以它的embedding vector，然后再相加

print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


## 1.2 计算输入序列中所有tokens的context vector

In [ ]:
'''1. Compute the un-normalized attention scores'''
attn_scores = torch.empty(inputs.shape[0], inputs.shape[0]) #这里之所以是inputs.shape[0]的原因是，对于每一个token，都要计算和其他所有token之间的注意力分数，而总的tokens的数量就是inputs.shape[0]，
# 也就是max_length的大小，输入序列越长，注意力分数的矩阵就越大，而且是二次方的增长关系，shape是max_length * max_length

#（1）使用循环的方式计算：速度慢，计算复杂度是O(n^2):
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)
print(attn_scores)

# (2) 使用矩阵相乘的方式计算：速度快
attn_scores = inputs @ inputs.T # @ 符号代表矩阵相乘的意思
print(attn_scores)

attn_scores = torch.matmul(inputs, inputs.T) #将inputs转置的根本原因是，
# 转置之后就能实现inputs中的每一行作为query，能够与inputs.T中的每一列相乘，作点积运算，得到最终的注意力分数结果矩阵
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])
tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])
tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6

In [ ]:
'''2. Apply normalization using softmax function'''
attn_weights = torch.softmax(attn_scores, dim = 1) # dim = 1是因为需要在列上进行计算
print(attn_weights)

print(attn_weights.sum(1)) #说明归一化成功了

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])
tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [ ]:
'''3. Comput the final context vector'''
all_context_vec = attn_weights @ inputs #这里不进行转置的原因是需要将attn_weights中的每一行的query的注意力权重，
# 分别乘到所有的token的embedding vector上，因此就不用转置了。
print(all_context_vec)

print(all_context_vec[1])
print(context_vec_2)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])
tensor([0.4419, 0.6515, 0.5683])
tensor([0.4419, 0.6515, 0.5683])


1.计算Context_vector过程的手动推导：
![](./images/计算上下文向量的过程推导.jpg)

# 2.包含训练权重的自注意力机制

In [1]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

## 2.1 这里以计算“journey”的context vector为例

In [ ]:
'''Step 1: Initialize the trainable weight matrix and get the querys, keys, values'''
# 1. 初始化可训练权重矩阵
x_2 = inputs[1]
d_in = inputs.shape[1] # the embedding size of the input token
d_out = 2 # 映射得到的可训练权重矩阵的embedding size，在LLM中，通常d_in = d_ou，这里是故意不一样的

print(x_2)

torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad = False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad = False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad = False)
# Note：这里是为展示，所以禁用了梯度计算，但是实际训练过程中，需要设置为True的
print(W_query)
print(W_key)
print(W_value)

# 2.计算投影后的querys， keys， values
# （1）“journey”的结果：
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value

print(query_2)
print(key_2)
print(value_2)

# (2) 对于所有输入序列中的tokens计算
print(inputs.shape)
print(inputs)

querys = inputs @ W_query
keys = inputs @ W_key
values = inputs @ W_value

print(querys)
print(keys)
print(values)
# 从输出的结果中可以看到对全部的tokens的结果中第2行的结果，与上面journey的结果一致，说明方法是对的

tensor([0.5500, 0.8700, 0.6600])
Parameter containing:
tensor([[0.2961, 0.5166],
        [0.2517, 0.6886],
        [0.0740, 0.8665]])
Parameter containing:
tensor([[0.1366, 0.1025],
        [0.1841, 0.7264],
        [0.3153, 0.6871]])
Parameter containing:
tensor([[0.0756, 0.1966],
        [0.3164, 0.4017],
        [0.1186, 0.8274]])
tensor([0.4306, 1.4551])
tensor([0.4433, 1.1419])
tensor([0.3951, 1.0037])
torch.Size([6, 3])
tensor([[0.4300, 0.1500, 0.8900],
        [0.5500, 0.8700, 0.6600],
        [0.5700, 0.8500, 0.6400],
        [0.2200, 0.5800, 0.3300],
        [0.7700, 0.2500, 0.1000],
        [0.0500, 0.8000, 0.5500]])
tensor([[0.2309, 1.0966],
        [0.4306, 1.4551],
        [0.4300, 1.4343],
        [0.2355, 0.7990],
        [0.2983, 0.6565],
        [0.2568, 1.0533]])
tensor([[0.3669, 0.7646],
        [0.4433, 1.1419],
        [0.4361, 1.1156],
        [0.2408, 0.6706],
        [0.1827, 0.3292],
        [0.3275, 0.9642]])
tensor([[0.1855, 0.8812],
        [0.3951, 1.0037],

In [ ]:
'''Step 2: 计算非归一化的注意力分数'''
# 1. 仅计算 journey 对于自身的注意力分数结果
keys_2 = keys[1]
attn_scores_22 = query_2.dot(keys_2) #都是一维向量，就不用转置了
# attn_scores_22 = torch.dot(query_2, keys_2) #结果和上面一行计算出来的一样
print(attn_scores_22)

# 2.计算 journey 对所有输入序列中tokens的注意力分数结果
attn_scores_2 = query_2 @ keys.T # 这里转置的原因和之前简化版的是一样的
print(attn_scores_2)

# 3.计算所有tokens之间的注意力分数
attn_scores = querys @ keys.T
print(attn_scores)

tensor(1.8524)
tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])
tensor([[0.9231, 1.3545, 1.3241, 0.7910, 0.4032, 1.1330],
        [1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440],
        [1.2544, 1.8284, 1.7877, 1.0654, 0.5508, 1.5238],
        [0.6973, 1.0167, 0.9941, 0.5925, 0.3061, 0.8475],
        [0.6114, 0.8819, 0.8626, 0.5121, 0.2707, 0.7307],
        [0.8995, 1.3165, 1.2871, 0.7682, 0.3937, 1.0996]])


In [13]:
'''Step 3: 对注意力分数进行缩放和归一化得到注意力权重'''
d_k = keys.shape[-1] # 用于缩放scale
# 1. 计算journey对自身的注意力权重
attn_weights_2 = torch.softmax(attn_scores_2 / d_k ** 0.5, dim = -1)

print(attn_weights_2)

# 2. 计算所有tokens之间的注意力权重
import numpy as np
attn_weights = torch.softmax(attn_scores / np.sqrt(d_k), dim = -1)
print(attn_weights)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])
tensor([[0.1551, 0.2104, 0.2059, 0.1413, 0.1074, 0.1799],
        [0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820],
        [0.1503, 0.2256, 0.2192, 0.1315, 0.0914, 0.1819],
        [0.1591, 0.1994, 0.1962, 0.1477, 0.1206, 0.1769],
        [0.1610, 0.1949, 0.1923, 0.1501, 0.1265, 0.1752],
        [0.1557, 0.2092, 0.2048, 0.1419, 0.1089, 0.1794]])


In [ ]:
'''Step 4: 计算得到最终的context vector'''
# 1.计算journey的context vector
context_vec_2 = attn_weights_2 @ values #这里不用转置的愿意就是让注意力权重乘到每一个value embedding上，然后求和
print(context_vec_2)

# 2. 计算所有token的context vectors 
context_vec = attn_weights @ values
print(context_vec) 

#这样就完成了从最开始tokenization和embedding之后的 token embedding，
# 到经过引入query, key, value权重矩阵的自注意力机制，
# 得到最终的context vectors

tensor([0.3061, 0.8210])
tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]])


# 3.将自注意力机制集成到一个类中

In [ ]:
'''1. 使用torch.nn.Parameter创建Q,K,V权重矩阵'''
import torch.nn as nn
class selfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        # 初始化可训练权重
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        # 使用权重矩阵将输入投影到Q,K,V
        queries = x @ self.W_query
        keys = x @ self.W_key
        values = x @ self.W_value

        #计算得到注意力分数
        attn_scores = queries @ keys.T

        # 缩放和归一化得到注意力权重
        attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim = -1)

        # 加权求和得到最终的context vector
        context_vec = attn_weights @ values

        return context_vec

In [17]:
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

d_in = inputs.shape[-1]
d_out = 2

torch.manual_seed(123)

sa_v1 = selfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [ ]:
'''2. 使用torch.nn.Linear()创建Q,K,V权重矩阵'''
import torch.nn as nn
class selfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias = False):
        super().__init__()
        # 初始化可训练权重
        self.W_query = nn.Linear(d_in, d_out, bias = qkv_bias)#这里将bias设置为False，那么Linear层的作用就相当于是简单的矩阵相乘了
        self.W_key = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias = qkv_bias)

    def forward(self, x):
        # 使用权重矩阵将输入投影到Q,K,V
        queries = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)

        #计算得到注意力分数
        attn_scores = queries @ keys.T

        # 缩放和归一化得到注意力权重
        attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim = -1)

        # 加权求和得到最终的context vector
        context_vec = attn_weights @ values

        return context_vec

In [20]:
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

d_in = inputs.shape[-1]
d_out = 2

torch.manual_seed(789)

sa_v2 = selfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


In [23]:
'''3. 让两种方法输出的结果一致'''
# 方法一：直接将v2的权重矩阵赋值给v1
sa_v1.W_query.data = sa_v2.W_query.weight.T
sa_v1.W_key.data = sa_v2.W_key.weight.T
sa_v1.W_value.data = sa_v2.W_value.weight.T

# 方法二：根据v2的权重的值，重新创建nn.Parameter对象
sa_v1.W_query = torch.nn.Parameter(sa_v2.W_query.weight.T)
sa_v1.W_key = torch.nn.Parameter(sa_v2.W_key.weight.T)
sa_v1.W_value = torch.nn.Parameter(sa_v2.W_value.weight.T)

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

d_in = inputs.shape[-1]
d_out = 2

print(sa_v1(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


Notes: __使用nn.Linear()的主要原因是它内置的的权重初始化方法更好，能够保证模型训练过程中的稳定。__